In [ ]:
#%% [markdown]
# # Advanced Portfolio Simulator Webapp
# 
# In this notebook, we integrate CS109 concepts such as Monte Carlo simulations, optimization via the efficient frontier, Bayesian updating, and sensitivity analysis. Users can input their desired stock tickers, date ranges, risk preferences, and simulation parameters through interactive widgets.

#%% [code]
# Imports & Basic Setup
import os
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy.optimize import minimize
from scipy import stats

# Set plotting style
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
plt.rcParams['figure.figsize'] = (18, 10)
np.random.seed(42)

#%% [markdown]
# ## User Input (Simulated via Variables for This Notebook)
# 
# In the actual webapp, these parameters would come from interactive widgets.
# 
# - **tickers_input:** Comma-separated tickers provided by the user.
# - **start_date & end_date:** Date pickers.
# - **num_portfolios:** Slider for simulation count.
# - **rf_rate:** Risk-free rate input.
# - **simulate_shock:** Checkbox for running a “what-if” shock analysis.

#%% [code]
# User-defined parameters (simulate webapp input)
tickers_input = "GOOG,AAPL,META,AMZN,MSFT,^GSPC"
start_date = "2012-05-18"
end_date   = "2023-01-01"
num_portfolios = 10000
rf_rate = 0.025
simulate_shock = True  # additional scenario simulation toggle

# Process tickers
tickers = [ticker.strip() for ticker in tickers_input.split(',')]

#%% [markdown]
# ## Data Download and Preprocessing
# 
# We retrieve the data using `yfinance` and compute daily returns.
# In a webapp, this step can include caching to speed up repeated queries.

#%% [code]
def download_stock_data(tickers, start, end):
    data = {}
    for ticker in tickers:
        df = yf.download(ticker, start=start, end=end, progress=False)
        if not df.empty:
            data[ticker] = df
    return data

# Download data for all tickers
stock_data = download_stock_data(tickers, start_date, end_date)

# Create a combined dataframe with closing prices
price_df = pd.concat([df['Close'] for df in stock_data.values()], axis=1)
price_df.columns = list(stock_data.keys())
print("Data Shape:", price_df.shape)
price_df.head()

#%% [markdown]
# ## Exploratory Data Analysis (EDA)
# 
# We visualize the closing prices and the distribution of returns.
# Users can select which companies to focus on via checkboxes (in the webapp).

#%% [code]
def plot_close_prices(df, tickers):
    plt.figure()
    for ticker in tickers:
        plt.plot(df.index, df[ticker], label=ticker)
    plt.xlabel("Date")
    plt.ylabel("Closing Price")
    plt.title("Daily Close Prices")
    plt.legend()
    plt.show()

plot_close_prices(price_df, tickers)

# Compute daily returns
returns_df = price_df.pct_change().dropna()

# Plot histogram and density for each stock return
def plot_return_distributions(returns_df, tickers):
    n = len(tickers)
    fig, axes = plt.subplots(nrows=(n+1)//2, ncols=2, figsize=(16, n*3))
    axes = axes.flatten()
    for i, ticker in enumerate(tickers):
        sns.histplot(returns_df[ticker], kde=True, ax=axes[i], stat="density", bins=50)
        axes[i].set_title(f"Return Distribution for {ticker}")
    plt.tight_layout()
    plt.show()

plot_return_distributions(returns_df, tickers)

#%% [markdown]
# ## Advanced CS109 Concepts: Bayesian Update for Expected Returns
# 
# As a new idea, we perform a Bayesian update on the expected returns.
# Suppose the prior belief for each stock's return is Normal, and new data (recent returns) are observed.
# This update can help refine the inputs for portfolio optimization.

#%% [code]
def bayesian_update(prior_mean, prior_std, data):
    """
    Update the prior Normal distribution with new data (sample mean and std).
    Assumes conjugate update (Normal-Normal).
    """
    n = len(data)
    sample_mean = np.mean(data)
    sample_var = np.var(data)
    
    # Precision (inverse variance)
    prior_prec = 1 / (prior_std**2)
    sample_prec = n / sample_var if sample_var > 0 else 0
    
    posterior_std = np.sqrt(1 / (prior_prec + sample_prec))
    posterior_mean = (prior_mean * prior_prec + sample_mean * sample_prec) * posterior_std**2
    return posterior_mean, posterior_std

# Example: Update for each ticker using the last 60 days of returns
bayesian_means = {}
for ticker in tickers:
    prior_mean = returns_df[ticker].mean()
    prior_std  = returns_df[ticker].std()
    recent_data = returns_df[ticker].tail(60)
    post_mean, post_std = bayesian_update(prior_mean, prior_std, recent_data)
    bayesian_means[ticker] = post_mean
    print(f"{ticker} Bayesian Updated Mean Return: {post_mean:.5f}, Std: {post_std:.5f}")

#%% [markdown]
# ## Portfolio Optimization Using Monte Carlo Simulation & Efficient Frontier
# 
# We simulate random portfolios using the Dirichlet distribution for weights.
# Additionally, we calculate the efficient frontier using optimization. Advanced CS109
# topics like convex optimization and risk metrics (e.g., Sharpe ratio) are highlighted.

#%% [code]
def calc_portfolio_perf(weights, mean_returns, cov, rf):
    portfolio_return = np.sum(mean_returns * weights) * 252  # annualized
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    return portfolio_return, portfolio_std, sharpe_ratio

def simulate_random_portfolios(num_portfolios, mean_returns, cov, rf):
    n_assets = len(mean_returns)
    results = np.zeros((num_portfolios, 3 + n_assets))
    for i in range(num_portfolios):
        weights = np.random.dirichlet(np.ones(n_assets), size=1).flatten()
        ret, vol, sr = calc_portfolio_perf(weights, mean_returns, cov, rf)
        results[i, :3] = [ret, vol, sr]
        results[i, 3:] = weights
    columns = ['Return', 'Volatility', 'Sharpe'] + list(mean_returns.index)
    return pd.DataFrame(results, columns=columns)

# Use Bayesian-updated returns for the mean return if desired,
# otherwise default to historical mean.
mean_returns = returns_df.mean()
bayesian_return_series = pd.Series(bayesian_means)

cov_matrix = returns_df.cov()

# Simulate portfolios
results_df = simulate_random_portfolios(num_portfolios, mean_returns, cov_matrix, rf_rate)
print("Simulated Portfolios:")
results_df.head()

# Plotting the efficient frontier from simulation
plt.figure()
plt.scatter(results_df.Volatility, results_df.Return, c=results_df.Sharpe, cmap='RdYlBu', marker='o', s=10)
plt.xlabel("Annualized Volatility")
plt.ylabel("Annualized Return")
plt.title("Random Portfolio Simulation & Efficient Frontier")
plt.colorbar(label='Sharpe Ratio')
plt.show()

#%% [markdown]
# ### Optimization Functions for Maximum Sharpe Ratio and Minimum Volatility
# 
# These functions use `scipy.optimize` to find optimal portfolios.

#%% [code]
def portfolio_volatility(weights, cov):
    return np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)

def portfolio_return(weights, mean_returns):
    return np.sum(mean_returns * weights) * 252

def negative_sharpe(weights, mean_returns, cov, rf):
    ret = portfolio_return(weights, mean_returns)
    vol = portfolio_volatility(weights, cov)
    return -(ret - rf) / vol

def optimize_portfolio(mean_returns, cov, rf):
    n_assets = len(mean_returns)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for _ in range(n_assets))
    init_guess = np.repeat(1/n_assets, n_assets)
    
    opt_sharpe = minimize(negative_sharpe, init_guess,
                          args=(mean_returns, cov, rf),
                          method='SLSQP', bounds=bounds, constraints=constraints)
    return opt_sharpe

optimum = optimize_portfolio(mean_returns, cov_matrix, rf_rate)
opt_weights = optimum.x
opt_return, opt_vol, opt_sharpe = calc_portfolio_perf(opt_weights, mean_returns, cov_matrix, rf_rate)
print("Optimal Portfolio (Max Sharpe):")
print("Return:", opt_return, "Volatility:", opt_vol, "Sharpe Ratio:", opt_sharpe)
print("Weights:", dict(zip(mean_returns.index, opt_weights)))

#%% [markdown]
# ## Sensitivity Analysis: What-If Market Shock Simulation
# 
# As an additional advanced feature, we simulate a market shock. The user can input a shock parameter (e.g., a percentage drop)
# and the system will adjust returns accordingly and re-calculate portfolio performance metrics.

#%% [code]
def simulate_market_shock(returns_df, shock_pct):
    """Apply a shock to the returns (e.g., -10% drop for a day)"""
    shocked_returns = returns_df.copy()
    # Here, we assume the shock applies uniformly to all assets on a specific day,
    # for demonstration we apply it to the most recent day.
    shocked_returns.iloc[-1] = shocked_returns.iloc[-1] * (1 + shock_pct)
    return shocked_returns

if simulate_shock:
    shock_pct = -0.10  # for example, a 10% drop in returns for the shock day
    shocked_returns = simulate_market_shock(returns_df, shock_pct)
    new_mean_returns = shocked_returns.mean()
    new_cov = shocked_returns.cov()
    # Re-run the optimization after shock
    opt_shock = optimize_portfolio(new_mean_returns, new_cov, rf_rate)
    shock_weights = opt_shock.x
    shock_ret, shock_vol, shock_sharpe = calc_portfolio_perf(shock_weights, new_mean_returns, new_cov, rf_rate)
    print("\nAfter Shock - Optimal Portfolio:")
    print("Return:", shock_ret, "Volatility:", shock_vol, "Sharpe Ratio:", shock_sharpe)
    print("Weights:", dict(zip(new_mean_returns.index, shock_weights)))

#%% [markdown]
# ## Conclusion and Next Steps
# 
# This notebook demonstrates advanced CS109 concepts including:
# - **Monte Carlo simulations** for portfolio generation.
# - **Optimization techniques** (convex optimization with SLSQP) for the efficient frontier.
# - **Bayesian updating** to refine expected returns.
# - **Sensitivity analysis** to assess portfolio performance under shock scenarios.
# 
# In a full webapp, interactive widgets would replace the hard-coded inputs and users could explore these models dynamically:
# - **Input panels** for selecting stocks and dates.
# - **Sliders** for simulation parameters (number of portfolios, risk-free rate, shock intensity).
# - **Real-time visualizations** that update as users adjust parameters.
#
# This approach not only highlights key CS109 probability and optimization topics but also connects them directly to real-world financial decision-making.

